## System configuration and validation

Before recording, we generate a configuration file which will tell `multicam_acquisition` what devices to use, how to trigger frame grabs, and how to write the results to disk. This notebook can also be used to verify before a recording session that the system is functioning correctly.

For generation of example configurations under several common hardware setups, see `notebooks/example-rigs/`.

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# TODO: refactor into end-user module?
from multicamera_acquisition.tests.writer.test_writers import (
    fps,
    n_test_frames,
)
from multicamera_acquisition.tests.interfaces.test_ir_cameras import (
    camera_type, 
    camera,
)
from multicamera_acquisition.interfaces.config import (
    partial_config_from_camera_list,
    create_full_camera_default_config,
)

from multicamera_acquisition.acquisition import (
    AcquisitionLoop
)

from multicamera_acquisition.paths import PROJECT_DIR

from multicamera_acquisition.config.config import load_config, save_config

from multicamera_acquisition.acquisition import (
    refactor_acquire_video,
    AcquisitionLoop
)
from pathlib import Path
from datetime import datetime
import os
from shutil import rmtree

### Set project paths

In [ ]:
config_path = PROJECT_DIR / "acquisition_config.yml"
tmp_directory = Path('/tmp/acq_test')

### Generate config

This section can be altered and run during system setup to create a configuration file for running acquisitions. Then the Run Test Acquisition section can be used as needed to check that chosen parameters result in satisfactory recordings.

If you have already generated a config file, you may skip this section as it will be loaded in Run Test Acquisition.

In [ ]:
# TODO: update with arduino config
# TODO: update with display config
# TODO: change example from continuous mode

# Generate camera config
camera_list = [
    {"name": "top", "brand": 'basler', "id": 0, "short_name": "continuous"},
    {"name": "bottom", "brand": 'basler', "id": 1, "short_name": "continuous"}
]
fps = 30

# Parse the "camera list" into a partial config
partial_new_config = partial_config_from_camera_list(camera_list, fps)

# Create the full config, filling in defaults where necessary
# This generates a default writer config for each camera as well. To specify
# non-default writers, see docs for `FFMPEG_Writer` or `NVC_Writer`.
full_config = create_full_camera_default_config(partial_new_config)

# Set up the acquisition loop section of the config
acq_config = AcquisitionLoop.default_acq_loop_config()
full_config["acq_loop"] = acq_config

In [ ]:
save_config(str(config_path), full_config)

### Run Test Acquisition

Load the configuration file pointed to by `config_path`, attempt to run a short acquisition, and visualize the results to validate system parameters.

In [7]:
# Load config
config = load_config(config_path)

In [ ]:
# Run short acquisition
curr_time = datetime.datetime.now().replace(microsecond=0).isoformat().replace(':', "-")
acq_config["max_frames_to_acqure"] = 30 # TODO: required for continuous cameras?
save_loc, full_config = refactor_acquire_video(
    tmp_directory / curr_time,
    config,
    recording_duration_s=1,
    append_datetime=True,
    overwrite=False,
)

In [ ]:
# Visualize results
# TODO: tap into visualization loop from `acquire_video`

In [ ]:
# Cleanup
for f in os.listdir(str(tmp_directory)):
    rmtree(tmp_directory / f)